In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import glob
from osgeo import gdal, ogr

In [8]:
def clip_raster_with_geojson(input_raster, output_clipped_raster, geojson_file):
    # Open the input raster
    input_ds = gdal.Open(input_raster)

    # Open the GeoJSON file
    geojson_ds = ogr.Open(geojson_file)
    layer = geojson_ds.GetLayer()

    # Create a new raster dataset for the clipped output
    clipped_ds = gdal.Warp(output_clipped_raster, input_ds, cutlineDSName=geojson_file, cropToCutline=True, dstNodata=0)

    # Close the datasets
    input_ds = None
    geojson_ds = None
    clipped_ds = None

In [9]:
# 文件路径和列表
image_path = '/content/drive/MyDrive/RS/S1_all_bands/'
json_path = '/content/drive/MyDrive/RS/S1_all_bands/sen1floods11_hand_labeled_label/'
image_list = glob.glob(image_path+'*.tif')
json_list = glob.glob(json_path+'*')
image_list = sorted(image_list)
json_list = sorted(json_list)[:-1]  # 除去最后一个整体的json文件
print(image_list)
print(json_list)

['/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000016384-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000032768-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/Ghana_ALL_Bands-0000000000-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/Ghana_ALL_Bands-0000000000-0000016384.tif', '/content/drive/MyDrive/RS/S1_all_bands/Ghana_ALL_Bands-0000016384-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/Ghana_ALL_Bands-0000016384-0000016384.tif', '/content/drive/MyDrive/RS/S1_all_bands/India_ALL_Bands-0000000000-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/India_ALL_Bands-0000000000-0000016384.tif', '/content/drive/MyDrive/RS/S1_all_bands/India_ALL_Bands-0000016384-0000000000.tif', '/content/drive/MyDrive/RS/S1_all_bands/India_ALL_Bands-0000016384-0000016384.tif', '/content/drive/MyDrive/RS/S1_all_bands/Mekong_ALL_Bands.tif', '/cont

In [10]:
for i in range(len(image_list)):
    raster_path = image_list[i]    # 栅格路径
    raster_ds = gdal.Open(raster_path)  # 栅格数据集
    geotransform = raster_ds.GetGeoTransform()
    # print(geotransform)
    min_x = geotransform[0]     # 栅格数据集范围
    max_x = geotransform[0] + geotransform[1] * raster_ds.RasterXSize
    min_y = geotransform[3] + geotransform[5] * raster_ds.RasterYSize
    max_y = geotransform[3]


    for j in range(len(json_list)):
        geojson_path = glob.glob(json_list[j]+'/*.json')[0]    # 矢量路径
        geojson_ds = ogr.Open(geojson_path)     # 矢量数据集
        geojson_layer = geojson_ds.GetLayer()
        geojson_extent = geojson_layer.GetExtent()      # 矢量数据集范围
        # print(geojson_extent)

        is_contained = (geojson_extent[0] >= min_x and  # 比较是否在范围内
                    geojson_extent[1] <= max_x and
                    geojson_extent[2] >= min_y and
                    geojson_extent[3] <= max_y)

        if is_contained:
            output_path = '/content/drive/MyDrive/RS/HAND/S1_split/' + os.listdir(json_list[j])[0][:-5] + '.tif'
            print(raster_path)
            print(output_path)
            clip_raster_with_geojson(raster_path, output_path, geojson_path)

/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_233925_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_242570_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_290290_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_294583_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_360519_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND/S1_split/Bolivia_60373_label.tif
/content/drive/MyDrive/RS/S1_all_bands/Bolivia_ALL_Bands-0000000000-0000000000.tif
/content/drive/MyDrive/RS/HAND